In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks', color_codes=True, font_scale=0.8)

import numpy as np
import pandas as pd
import scipy
from tqdm import tqdm

%matplotlib inline

In [3]:
# Re-preform the ysi-regression
ysi = pd.read_csv('ysi.csv').set_index('SMILES')
fragments = pd.read_csv('fragments.csv', index_col=0)

err = ysi.YSI_err**2

from fragdecomp.regression import BayesianRegressionOutlier
reg = BayesianRegressionOutlier(fragments.values, ysi.YSI, err, prior_sd=25)
sigma, beta, y_hat = reg.sample(1000)

/Users/pstjohn/Dropbox/nrel_documents/Papers/stitching-paper/model/fragdecomp/regression.py:38: UserWarning: X Matrix is not full-rank
  warn("X Matrix is not full-rank")


In [8]:
ysi[ysi.index.str.contains('#')]

,Species,CAS,Ref,Type,YSI,YSI_err
SMILES,,,,,,
C(#Cc1ccccc1)c1ccccc1,diphenylacetylene,501-65-5,2.0,aromatic,638.6,24.1
C#Cc1cc(C)ccc1C,"1-ethynyl-2,5-dimethylbenzene",74331-70-7,1.0,aromatic,512.7,19.6
C#Cc1ccccc1C,1-ethynyl-2-methylbenzene,766-47-2,1.0,aromatic,485.0,18.5
CCC#Cc1ccccc1,(1-butynyl)-benzene,622-76-4,1.0,aromatic,480.8,18.4
C#CCc1ccccc1,(2-propynyl)-benzene,10147-11-2,1.0,aromatic,443.7,17.1
C#Cc1ccc(CC)cc1,1-ethynyl-4-ethylbenzene,40307-11-7,1.0,aromatic,442.6,17.1
CC#Cc1ccccc1,(1-propynyl)-benzene,673-32-5,1.0,aromatic,427.5,16.5
C#Cc1cccc(C)c1,1-ethynyl-3-methylbenzene,766-82-5,1.0,aromatic,384.6,14.7
C#Cc1ccc(C)cc1,1-ethynyl-4-methylbenzene,766-97-2,1.0,aromatic,374.7,14.7


In [3]:
six_carbon_series = [
    'CCCCCCO', # 1-alcohol
    'CCCCC(O)C', # 2-alcohol
    'CCCC(C)(O)C', # 3-alcohol
    'CCCCCOC', # ester
    'CCCCCC(=O)O', # carboxylic acid
    'CCCCCC=O', # aldehyde
    'CCCCC(=O)C', # ketone
    'CCCCCC', # n-alkane
    'CCCC(C)C', # secondary branch
    'CCC(C)(C)C', # tertiary branch
    'CCCCC=C', # terminal alkene
    'CCCC=CC', # middle alkene
    'CCC=C(C)C', # branched alkene
    'CCCCC#C', # terminal alkyne,
    'CCCC#CC', # terminal alkyne,
    'C1CCCCC1', # ring
    'C1CCCC1C', # methyl ring
    'C1CCCC1=C', # alkene ring
]

benzene_series = [
    'c1ccccc1', # benzene
    'c1ccccc1C', # methyl
    'c1ccccc1CC', # ethyl
    'c1ccccc1C(=C)C', # alkene
    'c1ccccc1C#C', # alkyne
    'c1ccc2c(c1)CCCC2', # fused aliph ring
    'c1ccc2ccccc2c1', # fused aromatic ring
    'c1ccc(-c2ccccc2)cc1', # bipehynl
]

In [4]:
from fragdecomp.fragment_decomposition import get_fragments

In [5]:
frags_six = pd.Series(six_carbon_series).apply(get_fragments)
frags_six.index = six_carbon_series

frags_aro = pd.Series(benzene_series).apply(get_fragments)
frags_aro.index = benzene_series

assert frags_aro.columns.isin(fragments.columns).all()
assert frags_six.columns.isin(fragments.columns).all()

frags_six = frags_six.loc[:, fragments.columns].fillna(0.).astype(int)
frags_aro = frags_aro.loc[:, fragments.columns].fillna(0.).astype(int)

In [6]:
means_aro, hpd_aro = reg.predict(frags_aro, beta)
means_six, hpd_six = reg.predict(frags_six, beta)

In [7]:
six_carbon_series = np.array(six_carbon_series)[means_six.argsort()]
means_six = means_six[means_six.argsort()]
hpd_six = hpd_six[means_six.argsort()]

In [8]:
from rdkit.Chem.Draw import MolsToGridImage
from rdkit.Chem import AllChem
from rdkit.Chem import MolFromSmiles

In [9]:
from fragdecomp.fragment_decomposition import draw_mol_svg
from fragdecomp.chemical_conversions import canonicalize_smiles, get_iupac_name_from_smiles

In [10]:
from IPython.display import SVG

In [11]:
from itertools import chain

In [12]:
names_six = [get_iupac_name_from_smiles(smiles) for smiles in six_carbon_series]
names_aro = [get_iupac_name_from_smiles(smiles) for smiles in benzene_series]

In [13]:
names_six

['hexanoic acid',
 'hexanal',
 'hexan-2-one',
 '1-methoxypentane',
 'hexan-1-ol',
 'hexan-2-ol',
 'hexane',
 '2-methylpentan-2-ol',
 '2-methylpentane',
 'cyclohexane',
 'hex-1-ene',
 '2,2-dimethylbutane',
 'hex-2-ene',
 'methylcyclopentane',
 '2-methylpent-2-ene',
 'hex-1-yne',
 'hex-2-yne',
 'methylidenecyclopentane']

In [14]:
names_aro

['benzene',
 'toluene',
 'ethylbenzene',
 'prop-1-en-2-ylbenzene',
 'ethynylbenzene',
 '1,2,3,4-tetrahydronaphthalene',
 'naphthalene',
 "1,1'-biphenyl"]

In [15]:
means_aro

array([ 105.63542874,  177.07552182,  229.18162939,  311.58602241,
        318.32433564,  402.66909262,  525.63156346,  656.74519175])

In [16]:
hpd_aro.sum(1)/2

array([ 17.57880316,  13.48995062,  10.03139603,  52.39953642,
        25.04958958,  46.24094873,  26.95160952,  39.42170191])

In [17]:
mols = [MolFromSmiles(smiles) for smiles in benzene_series]

mol_align = MolFromSmiles('c1ccccc1')
AllChem.Compute2DCoords(mol_align)

for mol in mols:
    try:
        AllChem.GenerateDepictionMatching2DStructure(mol, mol_align)
    except ValueError:
        pass
    
aro_leg = ["{:.1f} &#xb1; {:.1f}".format(mean, sum(hpd)/2) for mean, hpd in zip(means_aro, hpd_aro)]

with open('fragment_images/aro_series.svg', 'w') as f:
    f.write(MolsToGridImage(mols, useSVG=True, subImgSize=(100, 100), molsPerRow=2, legends=aro_leg))

In [18]:
mol_align = MolFromSmiles('CCCCCC')
AllChem.Compute2DCoords(mol_align)

mols = [MolFromSmiles(smiles) for smiles in six_carbon_series]

# subms = [x for x in mols if x.HasSubstructMatch(mol_align)]
for mol in mols:
    if mol.GetAtoms()[0].IsInRing():
        continue
    try:
        AllChem.GenerateDepictionMatching2DStructure(mol, mol_align)
    except ValueError:
        pass

six_leg = ["{:.1f} &#xb1; {:.1f}".format(mean, sum(hpd)/2) for mean, hpd in zip(means_six, hpd_six)]

with open('fragment_images/six_series.svg', 'w') as f:
    f.write(MolsToGridImage(mols, useSVG=True, subImgSize=(100, 100), molsPerRow=3, legends=six_leg))